In [103]:
import numpy as np

In [104]:
class Node(): #creating nodes
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None): #initializating
        self.feature_index = feature_index #decision node
        self.threshold = threshold #decision node
        self.left = left #decision node
        self.right = right #decision node
        self.info_gain = info_gain #decision node
        self.value = value #leaf node
        
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2): #initializating
        self.root = None #root of the tree 
        self.min_samples_split = min_samples_split # stopping conditions
        self.max_depth = max_depth # stopping conditions
    def build_tree(self, dataset, current_depth=0): #building tree
        X, Y = dataset[:,:-1], dataset[:,-1]
        samples_numbers, features_numbers = np.shape(X)
        if samples_numbers>=self.min_samples_split and current_depth<=self.max_depth: # split until conditions suttisfied
            best_split = self.get_best_split(dataset, samples_numbers, features_numbers) # best split
            if best_split["info_gain"]>0:
                subtree_left = self.build_tree(best_split["left_dataset"], current_depth+1) # recur left
                subtree_right = self.build_tree(best_split["right_dataset"], current_depth+1) # recur right
                return Node(best_split["feature_index"], best_split["threshold"], # return decision node
                            subtree_left, subtree_right, best_split["info_gain"])
        leaf_value = self.calculate_leaf_value(Y) # compute leaf node
        return Node(value=leaf_value) 
    def split(self, dataset, feature_index, threshold): #splitting data      
        left_dataset = np.array([row for row in dataset if row[feature_index]<=threshold])
        right_dataset = np.array([row for row in dataset if row[feature_index]>threshold])
        return left_dataset, right_dataset
    def get_best_split(self, dataset, samples_numbers, features_numbers): #finding best split
        best_split = {}
        max_info_gain = -float("inf")
        for feature_index in range(features_numbers): #all features
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values) 
            for threshold in possible_thresholds: #present features
                left_dataset, right_dataset = self.split(dataset, feature_index, threshold) #current split
                if len(left_dataset)>0 and len(right_dataset)>0:
                    y, left_y, right_y = dataset[:, -1], left_dataset[:, -1], right_dataset[:, -1]
                    current_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    if current_info_gain>max_info_gain: # updating best split
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["left_dataset"] = left_dataset
                        best_split["right_dataset"] = right_dataset
                        best_split["info_gain"] = current_info_gain
                        max_info_gain = current_info_gain
        return best_split
    def print_tree(self, tree=None, indent=" "): #tree output printing
        if not tree:
            tree = self.root
        if tree.value is not None:
            print(tree.value)
        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)    
    def information_gain(self, parent, left_child, right_child, mode="entropy"): #computing information    
        weight_left = len(left_child) / len(parent)
        weight_right = len(right_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_left*self.gini_index(left_child) + weight_right*self.gini_index(right_child))
        else:
            gain = self.entropy(parent) - (weight_left*self.entropy(left_child) + weight_right*self.entropy(right_child))
        return gain
    def entropy(self, y): #computing entropy
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy   
    def gini_index(self, y): #computing gini index
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
    def calculate_leaf_value(self, Y): #computing leaf node
        Y = list(Y)
        return max(Y, key=Y.count)
    def make_prediction(self, x, tree): #making a prediction
        if tree.value!=None: return tree.value
        feature_value = x[tree.feature_index]
        if feature_value<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    def fit(self, X, Y): #training tree
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)  
    def predict(self, X): #predicting function
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

In [105]:
import pandas as pd
from sklearn.model_selection import train_test_split 
#I used sklearn only for train_test_split, not for tree decision

In [106]:
qs_csv = pd.read_csv("/home/kali/Desktop/data1.csv")
print(qs_csv.columns)
print("Size:")
print(qs_csv.shape)

Index(['Year', 'Rank', 'Name', 'Point', 'City', 'Country'], dtype='object')
Size:
(5250, 6)


In [107]:
qs_csv.isnull().sum()
qs_csv = qs_csv.dropna() 
qs_csv.isnull().sum()

Year       0
Rank       0
Name       0
Point      0
City       0
Country    0
dtype: int64

In [108]:
qs_csv_float  = qs_csv.drop(columns=['City', 'Country'], axis=1).set_index('Name') #Only float left
qs_csv_float

,Year,Rank,Point
Name,,,
Harvard University,2018,1,97.7
University of Cambridge,2018,2,94.6
University of Oxford,2018,2,94.6
Massachusetts Institute of Technology (MIT),2018,4,92.5
Johns Hopkins University,2018,5,92.1
...,...,...,...
National Cheng Kung University (NCKU),2022,346,60.7
University of New Mexico,2022,346,60.7
Universitas Indonesia,2022,348,60.6


In [109]:
X = qs_csv.iloc[:, :-1].values
Y = qs_csv.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)
print(X_train.shape)
print(Y_train.shape)

(3520, 5)
(3520, 1)


In [110]:
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train,Y_train)
classifier.print_tree()

X_4 <= Santa Barbara ? 0.010548607815731303
 left:X_1 <= 24 ? 0.011469684844478856
  left:X_4 <= Parkville ? 0.05794519822855837
    left:X_2 <= Harvard University ? 0.08763536171884256
        left: United States
        right: United Kingdom
    right: United States
  right:X_2 <= University of Wisconsin-Madison ? 0.011360435368795607
    left:X_2 <= University of Helsinki ? 0.016183274486490262
        left: United States
        right: United States
    right:X_2 <= Universität Regensburg ? 0.1263087271865767
        left: Germany
        right: France
 right:X_4 <= Seoul ? 0.026934224870175494
  left:X_4 <= Sendai City ? 0.306748398006408
    left:X_4 <= Santiago ? 0.3057094840116279
        left: Chile
        right: Japan
    right: South Korea
  right:X_4 <= St. Andrews ? 0.029293485598234525
    left:X_2 <= Shanghai Jiao Tong University ? 0.2592045911047346
        left: Singapore
        right: United Kingdom
    right:X_4 <= Taoyuan City ? 0.03450361354047615
        left: T

In [112]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings("ignore")
print("accuracy_score:", accuracy_score(Y_test,Y_pred))
print("confusion_matrix:\n", confusion_matrix(Y_test, Y_pred))
print("classification_report:\n", classification_report(Y_test, Y_pred))
# The result is almost similar as was in lab 1 but here it takes a lot of time to compute

accuracy_score: 0.32471835652750164
confusion_matrix:
 [[  0   0   0 ...   6   0   0]
 [  0   0   0 ...  64   0   0]
 [  0   0   0 ...  15   0   0]
 ...
 [  0   0   0 ... 346   0   0]
 [  0   0   0 ...  50   0   0]
 [  0   0   0 ...  19   0   0]]
classification_report:
                  precision    recall  f1-score   support

      Argentina       0.00      0.00      0.00         6
      Australia       0.00      0.00      0.00        77
        Austria       0.00      0.00      0.00        15
        Belgium       0.00      0.00      0.00        32
         Brazil       0.00      0.00      0.00        24
         Canada       0.00      0.00      0.00        63
          Chile       1.00      1.00      1.00         9
 Czech Republic       0.00      0.00      0.00         2
        Czechia       0.00      0.00      0.00         1
        Denmark       0.00      0.00      0.00        19
          Egypt       0.00      0.00      0.00         5
        Finland       0.00      0.00      0.